In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from CRUD import AnimalShelter

# Fixing column headers creates a problem where columns size to data but not to headers, which
# makes some column headers unreadable and sloppy.  This function generates a custom style to manage
# column widths based on the length of data in each column.

# Credit: https://github.com/plotly/dash-table/issues/432

def create_conditional_style(df):
    PADDING = 24
    PIXEL_FOR_CHAR = 6
    style=[]
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                pass
        col_list = df[col].values.tolist()
        col_list = [s if type(s) is str else str(s) for s in col_list]
        col_list.append(col)
        name_length = len(max(col_list, key=len))
        pixel = PADDING + round(name_length*PIXEL_FOR_CHAR)
        pixel = str(pixel) + 'px'
        if pd.api.types.infer_dtype(df[col]) == 'string' or pd.api.types.infer_dtype(df[col]) == 'boolean' and not pd.api.types.is_datetime64_any_dtype(df[col]):
            style.append({'if': {'column_id': col}, 'minWidth': pixel, 'textAlign': 'left'})
        else:
            style.append({'if': {'column_id': col}, 'minWidth': pixel})
    return style


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "barkwoof"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read_documents({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Joe Clancy - 6/17/2024 - CS340 - Project 2'))),
    html.Hr(),
    html.Center(html.A(href="http://www.snhu.edu",children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='200', width='200')])),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    html.P('Filter for candidate animals:'),
    dcc.Dropdown(id='filter-type',
                 options=[{'label':'Water Rescue', 'value': 'water'},
                          {'label':'Mountain or Wilderness Rescue', 'value': 'mountain'},
                          {'label':'Disaster or Individual Tracking', 'value': 'disaster'},
                          {'label':'No Filter', 'value': 'reset'}],
                 value='reset'),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single', # Allow one row to be selectable
                         style_cell_conditional=create_conditional_style(df), # Apply custom column widths
                         sort_action='native', # Enable sorting by column
                         sort_mode='single', # Only sort by one column at a time
                         fixed_rows={'headers':True}, # Fix column headers
                         selected_rows = [0], # Start with first row selected
                         page_size = 10, # Set maximum number of rows
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
            ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if not filter_type:
        return
    
    if filter_type == 'water':
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "Labrador"}},
             {"breed":{"$regex": "Chesa Bay"}},
             {"breed":{"$regex": "Newfoundland"}}], 
         "sex_upon_outcome":"Intact Female",
         "age_upon_outcome_in_weeks":{"$gte":26},
         "age_upon_outcome_in_weeks":{"$lte":156}}
        
    elif filter_type == 'mountain':       
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "German Shepherd"}},
             {"breed":{"$regex": "Alaskan Malamute"}},
             {"breed":{"$regex": "Siberian Husky"}},
             {"breed":{"$regex": "Rottweiler"}},
             {"breed":{"$regex": "Old English Sheepdog"}}], 
         "sex_upon_outcome":"Intact Male",
         "age_upon_outcome_in_weeks":{"$gte":26},
         "age_upon_outcome_in_weeks":{"$lte":156}}
        
    elif filter_type == 'disaster':        
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "German Shepherd"}},
             {"breed":{"$regex": "Doberman Pinscher"}},
             {"breed":{"$regex": "Golden Retriever"}},
             {"breed":{"$regex": "Rottweiler"}},
             {"breed":{"$regex": "Bloodhound"}}], 
         "sex_upon_outcome":"Intact Male",
         "age_upon_outcome_in_weeks":{"$gte":20},
         "age_upon_outcome_in_weeks":{"$lte":300}}
        
    else:
        query = {}
    
    df = pd.DataFrame.from_records(db.read_documents(query))
    df.drop(columns=['_id'],inplace=True)

    data=df.to_dict('records')
    
    return data, [0]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    #[Input('datatable-id', "derived_viewport_data")])
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    if not viewData:
        return

    dff = pd.DataFrame.from_dict(viewData)    
    
    return [
        dcc.Graph(
            id = 'pie_chart',            
            figure = px.pie(dff, names='breed')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    if not selected_columns:
        return
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '750px', 'height': '500px'}, center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:29127/
